In [1]:
import sys
sys.path.append('../../')

In [2]:
from codes.docs.analysis import graph_analysis, data_preprocessing

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline


In [4]:
obsv_p_value=pd.read_csv('./preprocessed_data/diffusion/term/observed_univariate_test.csv')

In [98]:
obsv_p_value

,Connection,prs_pval,prs_coef,GA_pval,GA_coef,PMA_pval,PMA_coef,Gender_pval,Gender_coef,Adj_R_square,model_pval,PRS_threshold
0,PreCG.R_PreCG.L,0.250443,0.438658,0.434077,0.002754,5.145775e-01,0.001560,0.691335,-0.002655,0.003581,3.411659e-01,X1e.08
1,SFGdor.L_PreCG.L,0.761467,-1.576926,0.883218,0.007046,2.154667e-02,0.075665,0.180319,-0.122539,0.039043,4.132944e-02,X1e.08
2,SFGdor.L_PreCG.R,0.350050,0.162456,0.707692,-0.000601,4.422449e-01,-0.000839,0.210553,-0.003826,0.002123,3.678854e-01,X1e.08
3,SFGdor.R_PreCG.L,0.119149,0.377766,0.073799,0.004008,3.934512e-01,-0.001296,0.936013,-0.000340,0.017708,1.553779e-01,X1e.08
4,SFGdor.R_PreCG.R,0.768330,-1.828920,0.645865,-0.026355,3.124186e-03,0.116972,0.811488,-0.025997,0.048545,2.206321e-02,X1e.08
...,...,...,...,...,...,...,...,...,...,...,...,...
27583,ITG.R_HES.R,0.468726,-25.166730,0.402139,0.000922,7.155907e-03,-0.002052,0.000157,-0.008159,0.124433,8.350859e-05,X1
27584,ITG.R_STG.R,0.384304,-623.172000,0.620042,0.011239,1.275813e-01,-0.023762,0.000476,-0.154915,0.078251,2.768626e-03,X1
27585,ITG.R_TPOsup.R,0.331244,-907.061100,0.782151,0.008164,4.939543e-02,-0.040033,0.841575,0.011311,0.015543,1.762710e-01,X1
27586,ITG.R_MTG.R,0.133132,-5583.840000,0.273121,0.128825,2.541545e-08,0.472438,0.004447,-0.648445,0.311347,3.027523e-12,X1


In [5]:
ROIs_combinations=data_preprocessing.ROIs_combinations('./dataset/Regions_of_interests.csv')

In [135]:
def calculate_biggest_network_size(G,retain_the_component_sizes=False):
    """
    Return the size (number of edges) of the largest connected component graph.
    Argument
        G: a networkx Graph.
    Return
        biggest_size: size of the largest connected component.
    """
    component_sizes=np.asarray([G.subgraph(c).copy().number_of_edges() for c in nx.connected_components(G)])
    biggest_size=np.max(component_sizes)
    if retain_the_component_sizes:
        return component_sizes
    return biggest_size

In [133]:
(np.asarray([testG.subgraph(c).copy().number_of_edges() for c in nx.connected_components(testG)]))

array([ 0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  2,  0,
       10,  4,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0])

In [127]:
PRS_threshold=['X1e.08','X1e.07','X1e.06','X1e.05','X0.0001','X0.001','X0.01','X0.05','X0.1','X0.5','X1']
obsv_component_size_distribution=dict()

for threshold in tqdm.tqdm(PRS_threshold):

    obsv_df=obsv_p_value.loc[obsv_p_value['PRS_threshold']==threshold,('Connection','prs_pval')]
    
    obsv_matrix=np.ones(ROIs_combinations.shape)
    for i in range(len(obsv_df['Connection'])):
        i1,j1=np.where(ROIs_combinations==np.asarray(obsv_df['Connection'])[i])
        obsv_matrix[i1,j1]=obsv_matrix[j1,i1]=np.asarray(obsv_df['prs_pval'])[i]
        thresholded_matrix=np.where(obsv_matrix<0.01,1,0)
    obsv_component_size_distribution[threshold]=calculate_biggest_network_size(nx.from_numpy_array(thresholded_matrix))

100%|██████████| 11/11 [00:04<00:00,  2.27it/s]


In [128]:
obsv_component_size_distribution

{'X1e.08': 10,
 'X1e.07': 9,
 'X1e.06': 8,
 'X1e.05': 5,
 'X0.0001': 6,
 'X0.001': 5,
 'X0.01': 3,
 'X0.05': 4,
 'X0.1': 8,
 'X0.5': 8,
 'X1': 5}

In [120]:
import glob
perm=glob.glob('./log/diffusion/term/perm_data_in_R/*')[0]

In [30]:
import tqdm

In [138]:
perm_component_size_distribution=dict()
n=0
for perm in tqdm.tqdm(glob.glob('./log/diffusion/term/perm_data_in_R/*')):
    perm_component_size_distribution[n]=dict()
    perm_p_value=pd.read_csv(perm)
    for threshold in PRS_threshold:    
        perm_df=perm_p_value.loc[perm_p_value['PRS_threshold']==threshold,('Connection','prs_pval')]
        perm_matrix=np.ones(ROIs_combinations.shape)
        for i in range(len(perm_df['Connection'])):
            i1,j1=np.where(ROIs_combinations==np.asarray(perm_df['Connection'])[i])
            perm_matrix[i1,j1]=perm_matrix[j1,i1]=np.asarray(perm_df['prs_pval'])[i]
        thresholded_matrix_perm=np.where(perm_matrix<0.01,1,0)
        perm_component_size_distribution[n][threshold]=calculate_biggest_network_size(nx.from_numpy_array(thresholded_matrix_perm),retain_the_component_sizes=True)
    n+=1


100%|██████████| 1000/1000 [1:06:26<00:00,  3.99s/it]


In [139]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [142]:
save_obj(perm_component_size_distribution,'./log/diffusion/term/perm_component_size_distribution_001')

In [7]:
import pickle
with open('./log/diffusion/term/perm_component_size_distribution_001.pkl', 'rb') as handle:
    perm_component_size_distribution = pickle.load(handle)

In [26]:
PRS_threshold=['X1e.08','X1e.07','X1e.06','X1e.05','X0.0001','X0.001','X0.01','X0.05','X0.1','X0.5','X1']
obsv_component_size={'X1e.08': 10,
 'X1e.07': 9,
 'X1e.06': 8,
 'X1e.05': 5,
 'X0.0001': 6,
 'X0.001': 5,
 'X0.01': 3,
 'X0.05': 4,
 'X0.1': 8,
 'X0.5': 8,
 'X1': 5}
for threshold in PRS_threshold:
    X1e08_distribution=[]
    for n in range(1000):
        X1e08_distribution.append(np.max(perm_component_size_distribution[n][threshold]))
    print(f'pvalue for threshold {threshold} is {len(np.where(np.asarray(X1e08_distribution)>obsv_component_size[threshold])[0])/1000}')

pvalue for threshold X1e.08 is 0.448
pvalue for threshold X1e.07 is 0.483
pvalue for threshold X1e.06 is 0.526
pvalue for threshold X1e.05 is 0.704
pvalue for threshold X0.0001 is 0.656
pvalue for threshold X0.001 is 0.701
pvalue for threshold X0.01 is 0.862
pvalue for threshold X0.05 is 0.779
pvalue for threshold X0.1 is 0.554
pvalue for threshold X0.5 is 0.555
pvalue for threshold X1 is 0.706


In [22]:
print(f'pvalue for threshold {threshold} is {len(np.where(np.asarray(X1e08_distribution)>10)[0])/1000}')

0.448
